In [ ]:
# default_exp datasets.genetic_datasets

# module name here

> API details.

In [ ]:
#hide
from nbdev.showdoc import *
import pandas as pd
from kedro.config import ConfigLoader
from kedro.io import DataCatalog
from dataclasses import dataclass
import itertools

/Users/ahoang/Documents/Learning/nbdev_tutorial_2nd_try/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
#export

from typing import Any, Dict, List, Optional, Literal, Union
from enum import Enum
import numpy as np
from kedro.io import AbstractVersionedDataSet
from pydantic import BaseModel
from pathlib import Path, PosixPath
from pydantic.dataclasses import dataclass
from dataclasses import InitVar, asdict
from copy import deepcopy
import fsspec
from kedro.io.core import (
    AbstractVersionedDataSet,
    DataSetError,
    Version,
    get_filepath_str,
    get_protocol_and_path,
)
from kedro.extras.datasets.pandas import CSVDataSet
from fastcore.meta import delegates
from functools import partial, wraps, lru_cache
from types import SimpleNamespace
from tqdm.auto import tqdm
import pandas as pd

In [ ]:
#hide

def print_with_space(*args, **kwargs):
    print(*args, **kwargs)
    print("------")


/Users/ahoang/Documents/Learning/nbdev_tutorial_2nd_try/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
conf_loader = ConfigLoader("conf/base")
conf_catalog = conf_loader.get("catalog*.yaml")
catalog = DataCatalog.from_config(conf_catalog)
gen_file_path = "data/test_data/gen_file/test_CASE_MS_chr22.gen"

In [ ]:
catalog.list()

['genetic_file',
 'genetic_file_common_folder',
 'genetic_file_single',
 'geno_each_sample',
 'case_geno_each_sample',
 'control_geno_each_sample']

In [ ]:
catalog.load("genetic_file")._file_path.full_file_path

{'case': Path('data/test_data/gen_file/test_CASE_MS_chr22.gen'),
 'control': Path('data/test_data/gen_file/test_CONTROL_MS_chr22.gen')}

In [ ]:
test_gen = pd.read_csv(gen_file_path,
            sep= " ",
            usecols = [1],
            names = ["rsid"],
            header= None)
test_gen

,rsid
0,rs77948203
1,rs1014626
2,rs9610458
3,rs5762201
4,rs1004237
5,rs134490
6,rs4821519
7,rs1003500
8,rs5756405


In [ ]:
test_gen["rsid"].tolist()

/Users/ahoang/Documents/Learning/nbdev_tutorial_2nd_try/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['rs77948203',
 'rs1014626',
 'rs9610458',
 'rs5762201',
 'rs1004237',
 'rs134490',
 'rs4821519',
 'rs1003500',
 'rs5756405']

In [ ]:
query_rsids = ["rs77948203", "rs134490"]
query_rsids

['rs77948203', 'rs134490']

In [ ]:
query_row_index_list = test_gen.query("rsid == @query_rsids").index.tolist()
query_row_index_list

[0, 5]

In [ ]:
test = pd.read_csv(gen_file_path,
            sep= " ",
            skiprows= lambda x: x not in query_row_index_list,
            #iterator = True,
            chunksize = 1,
            header= None)

In [ ]:
next(pd.read_csv(gen_file_path,
            sep= " ",
            skiprows= lambda x: x not in query_row_index_list,
            #iterator = True,
            chunksize = 1,
            header= None))

,0,1,2,3,4,5,6,7,8,9,...,29311,29312,29313,29314,29315,29316,29317,29318,29319,29320
0,---,rs77948203,21249165,G,A,1,0.0,0.0,1,0.0,...,0,1,0,0,1,0.0,0.0,1,0,0


In [ ]:
@dataclass
class GenFile:
    filename: str
    rsid_col: int
    snp_col: int
    snps: List[str]
    
    

/Users/ahoang/Documents/Learning/nbdev_tutorial_2nd_try/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
pd.read_csv(gen_file_path,
            sep= " ",
            usecols = [1],
            names = ["rsid"],
            skiprows= lambda x: x not in query_row_index_list,
            iterator = True,
            header= None).f

'data/test_data/gen_file/test_CASE_MS_chr22.gen'

In [ ]:
#export
class SingleFilePathSchema(BaseModel):
    folder: str
    full_file_name: str
    file_name: Optional[str] = None
    extension: Optional[str] = None
    split_by_chromosome: Optional[bool] = None
    
    def __init__(self, **data: Any):
        super().__init__(**data)
        if self.file_name is None or self.extension is None:
            self.file_name, *_, self.extension = self.full_file_name.split(".")
            
    def get_full_file_path(self, chrom:Optional[int]=None):
        if self.split_by_chromosome and chrom is None:
            raise ValueError("Need chrom number")
        else:
            formatted_file_name = self.full_file_name.format(chrom_num=chrom)
            return (Path(self.folder)/formatted_file_name)
    
    @property
    def full_file_path(self):
        return self.get_full_file_path()
    
    @property
    def protocol_and_path(self):
        return get_protocol_and_path(self.full_file_path.as_posix())
    
    @property
    def protocol(self):
        return self.protocol_and_path[0]
    
    #validate full file name when split by chrom here

@dataclass
class CaseControlFilePathSchema():
    case: SingleFilePathSchema
    control: SingleFilePathSchema
    common_folder : InitVar(Optional[str]) = None
    
    def __getattr__(self, attr, **kwargs):
        return {
            k: getattr(v, attr) for k, v in self.to_dict().items()
        }

    def __post_init__(self, common_folder):
        if common_folder is not None:
            self.case = SingleFilePathSchema(folder=common_folder, **self.case)
            self.control = SingleFilePathSchema(folder=common_folder, **self.control)
    
    @property
    def protocol(self):
        if self.case.protocol != self.control.protocol:
            raise ValueError(f"Currently only the same file system for case and control file is supported.\n Case is located in {self.case.protocol} system. Control is located in {self.control.protocol} ")
        return self.case.protocol
    
    def to_dict(self):
        return asdict(self)
    
    def apply_func(self, func, **kwargs):
        print(func)
        print(self.to_dict().items())
        return {
            k: func(v, **kwargs) for k, v in self.to_dict().items()
        }
#     def __post_init_post_parse__(self, common_folder):
#         self.protocol, _ = self.case.protocol_and_path

            

In [ ]:
catalog.load("genetic_file_common_folder").full_file_path

{'case': Path('data/test_data/gen_file/test_CASE_MS_chr22.gen'),
 'control': Path('data/test_data/gen_file/test_CONTROL_MS_chr22.gen')}

In [ ]:
conf_catalog["genetic_file_single"]["file_path"]

{'folder': 'data/test_data/gen_file',
 'full_file_name': 'test_CASE_MS_chr22.gen'}

In [ ]:
test_genetic_file_single_file_path = SingleFilePathSchema(**conf_catalog["genetic_file_single"]["file_path"])
assert test_genetic_file_single_file_path.full_file_path == Path("data/test_data/gen_file/test_CASE_MS_chr22.gen")
assert test_genetic_file_single_file_path.protocol == "file"

In [ ]:
#export


class MissingAttributeError(Exception):
    pass

def requires(*required_attrs):        
    def wrapper(method):

        @functools.wraps(method)
        def inner_wrapper(self, *args, **kargs):
            if not all(hasattr(self, attr) for attr in required_attrs):
                raise MissingAttributeError()
            return method(self, *args, **kargs)

        return inner_wrapper
    return wrapper

class GenFileFormat(BaseModel):
    filepath: Union[str, Path]
    prob_n_cols: int
    initial_cols: List[str]
    rsid_col: str
    ref_col: str
    alt_col: str
    ref_alt_delim: Optional[str]
    pandas_args: Dict[str, Any]
    sample_ids: Optional[str] 
    
    # check that num_cols - initial cols is divisible by prob_n_cols
    # if ref col == alt col then ref_alt_delim needs to be specified, so ref,alt = ref_col.str.split(delim)
    
    def col_name_generator(self, include_initial_cols=True):
        
        if include_initial_cols:
            for col in self.initial_cols:
                yield col
        
        if not self.sample_ids:
            sample_ids_gen = (f"sample{i}" for i in itertools.count(1))
        else:
            sample_ids_gen = (sample_id for sample_id in sample_ids)
        
        while True:
            cur_sample_id = next(sample_ids_gen)
            for allele_prob_col_num in range(1, self.prob_n_cols + 1):
                yield f"{cur_sample_id}_{allele_prob_col_num}"
    
    @property
    def gen_info_cols(self):
        return [self.rsid_col, self.ref_col, self.alt_col]
    
    @property
    def column_headers(self):
        #read the first non-header line
        first_line_df = next(self._load_unprocessed(chunksize=1))
        generator = self.col_name_generator()
        return [next(generator) for col in first_line_df.columns]
    
    @property
    def sample_cols(self):
        cols = [col for col in self.column_headers if col not in set(self.initial_cols)]
        return cols
    
    @property
    def _sample_list(self):
        if self.sample_ids is not None:
            return self.sample_ids
        else:
            return sorted(list(set([col.split("_")[0] for col in self.sample_cols])))
        
    @property
    def num_samples(self):
        return len(self.sample_cols)/self.prob_n_cols
        
        
    @property
    def load_args(self):
        load_args = deepcopy(self.pandas_args)
        col_names = self.column_headers
        load_args["names"] = col_names
        return load_args
    
    @delegates(pd.read_csv)
    def _load_unprocessed(self, **kwargs):
        return pd.read_csv(filepath_or_buffer = self.filepath, **self.pandas_args, **kwargs)
    
    
    @delegates(pd.read_csv)
    def load(self, **kwargs):
        df = pd.read_csv(filepath_or_buffer = self.filepath, **self.load_args, **kwargs)
        #df.columns = self.column_headers
        return df
    
    
    @delegates(pd.read_csv)
    def get_rsid_df(self, rsid_list=None, **kwargs):
        rsid_df = self.load(usecols = [self.rsid_col])
        if rsid_list is not None:
            found_rsid_df = rsid_df.query(f"{self.rsid_col} in @rsid_list")
        else:
            found_rsid_df = rsid_df
        found_index = found_rsid_df.index
        found_rsids = set(found_rsid_df[self.rsid_col].unique())
        found_rsid_df_full = self.load(skiprows = lambda x: x not in found_index, **kwargs)
        return found_rsid_df_full
    
    @delegates(load)
    def get_genotypes_df(self, rsid_list: List = None, **kwargs):
        if rsid_list is not None:
            geno_df = self.get_rsid_df(rsid_list = rsid_list, usecols = self.gen_info_cols, **kwargs)
        else:
            geno_df = self.load(usecols = self.gen_info_cols,**kwargs)
        geno_df["homo_ref"] = geno_df.loc[:,[self.ref_col]] *2
        make_het_geno_func = lambda row: ''.join(sorted([row[self.ref_col],
                                   row[self.alt_col]]))
        geno_df["het"] = geno_df[[self.ref_col, self.alt_col]].apply(make_het_geno_func, axis=1)
        geno_df["homo_alt"] = geno_df.loc[:,self.alt_col] *2
        
        geno_df = geno_df.set_index(self.rsid_col)
        return geno_df
    
    
    def sample_columns_iter(self, rsid_list=None, **kwargs):
        
        df = self.get_rsid_df(rsid_list = rsid_list, **kwargs)
        df = df.set_index(self.rsid_col)
        
        for sample in self._sample_list:
            sample_cols = [f"{sample}_{allele_prob_col_num}" for allele_prob_col_num in range(1, self.prob_n_cols + 1)]
            sample_cols_df = df[sample_cols]
            yield SimpleNamespace(sample_id = sample,
                                  sample_df = sample_cols_df)
            
    
    #the function can accept sample_id kwarg
    def apply_func_to_all_samples(self, func, rsid_list=None, **kwargs):
        result_dict = [func(sample_obj.sample_df, sample_id = sample_obj.sample_id, **kwargs) for sample_obj in tqdm(self.sample_columns_iter(rsid_list=rsid_list))]#{sample_obj.sample_id: func(sample_obj.sample_df, sample_id = sample_obj.sample_id, **kwargs) for sample_obj in tqdm(self.sample_columns_iter(rsid_list=rsid_list))}
        result_df = pd.concat(result_dict, axis=1)
        #, orient="records")
        return result_df
    
#     @property
#     def single_line_iter(self, **runtime_kwargs):
#         return pd.read_csv(self.filepath,
#                            **self.load_args,
#                            **runtime_kwargs,
#                            chunksize=1)
    
#     #@delegates(pd.read_csv, but= list(self.load_args.keys()))
#     def load_full(self, **kwargs):
#         return pd.read_csv(self.filepath,
#                            **self.load_args,
#                           **kwargs)
    
#     #if ref_col = alt_col then ref_alt_delim need to be specified
    

In [ ]:
conf_catalog["genetic_file"]["load_args"]

{'prob_n_cols': 3,
 'initial_cols': ['dashes', 'rsid', 'position', 'ref', 'alt'],
 'rsid_col': 'rsid',
 'ref_col': 'ref',
 'alt_col': 'alt',
 'pandas_args': {'sep': ' ', 'header': None}}

In [ ]:
test_genetic_file_single_file_path.full_file_path

Path('data/test_data/gen_file/test_CASE_MS_chr22.gen')

In [ ]:
test_genfile_format = GenFileFormat(filepath = test_genetic_file_single_file_path.full_file_path,
                                    **conf_catalog["genetic_file"]["load_args"])
test_genfile_format

GenFileFormat(filepath=Path('data/test_data/gen_file/test_CASE_MS_chr22.gen'), prob_n_cols=3, initial_cols=['dashes', 'rsid', 'position', 'ref', 'alt'], rsid_col='rsid', ref_col='ref', alt_col='alt', ref_alt_delim=None, pandas_args={'sep': ' ', 'header': None}, sample_ids=None)

In [ ]:
test_genfile_format.load() #genotypes()

,dashes,rsid,position,ref,alt,sample1_1,sample1_2,sample1_3,sample2_1,sample2_2,...,sample9769_3,sample9770_1,sample9770_2,sample9770_3,sample9771_1,sample9771_2,sample9771_3,sample9772_1,sample9772_2,sample9772_3
0,---,rs77948203,21249165,G,A,1,0.000,0.000,1,0.000,...,0,1,0,0,1,0.000,0.000,1,0,0
1,---,rs1014626,21461017,C,T,0,0.000,1.000,0,0.000,...,1,0,0,1,0,0.000,1.000,0,0,1
2,---,rs9610458,22205353,C,T,0,0.000,1.000,0,0.000,...,0,0,1,0,0,1.000,0.000,0,0,1
3,---,rs5762201,27888455,A,G,0,0.000,1.000,0,0.012,...,1,0,0,1,0,0.000,1.000,0,0,1
4,---,rs1004237,28068501,C,T,1,0.000,0.000,1,0.000,...,0,1,0,0,1,0.000,0.000,1,0,0
5,---,rs134490,28730175,C,T,0,0.232,0.768,0,0.014,...,0,0,1,0,0,0.356,0.644,0,0,1
6,---,rs4821519,37102100,G,C,1,0.000,0.000,0,1.000,...,0,1,0,0,1,0.000,0.000,1,0,0
7,---,rs1003500,37262769,C,T,1,0.000,0.000,1,0.000,...,0,1,0,0,1,0.000,0.000,1,0,0
8,---,rs5756405,37310954,A,G,0,1.000,0.000,1,0.000,...,1,0,1,0,1,0.000,0.000,0,1,0


In [ ]:
test_header = test_genfile_format.column_headers
assert test_header[-1] == 'sample9772_3'

/Users/ahoang/Documents/Learning/nbdev_tutorial_2nd_try/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
test_genfile_format.get_rsid_df(["rs77948203", "rs9610458"])

,dashes,rsid,position,ref,alt,sample1_1,sample1_2,sample1_3,sample2_1,sample2_2,...,sample9769_3,sample9770_1,sample9770_2,sample9770_3,sample9771_1,sample9771_2,sample9771_3,sample9772_1,sample9772_2,sample9772_3
0,---,rs77948203,21249165,G,A,1,0.0,0.0,1,0.0,...,0,1,0,0,1,0.0,0.0,1,0,0
1,---,rs9610458,22205353,C,T,0,0.0,1.0,0,0.0,...,0,0,1,0,0,1.0,0.0,0,0,1


In [ ]:
next(test_genfile_format.get_rsid_df(["rs77948203", "rs9610458"], chunksize=1))

/Users/ahoang/Documents/Learning/nbdev_tutorial_2nd_try/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,dashes,rsid,position,ref,alt,sample1_1,sample1_2,sample1_3,sample2_1,sample2_2,...,sample9769_3,sample9770_1,sample9770_2,sample9770_3,sample9771_1,sample9771_2,sample9771_3,sample9772_1,sample9772_2,sample9772_3
0,---,rs77948203,21249165,G,A,1,0.0,0.0,1,0.0,...,0,1,0,0,1,0.0,0.0,1,0,0


In [ ]:
test_genfile_format.get_genotypes_df()

/Users/ahoang/Documents/Learning/nbdev_tutorial_2nd_try/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,ref,alt,homo_ref,het,homo_alt
rsid,,,,,
rs77948203,G,A,GG,AG,AA
rs1014626,C,T,CC,CT,TT
rs9610458,C,T,CC,CT,TT
rs5762201,A,G,AA,AG,GG
rs1004237,C,T,CC,CT,TT
rs134490,C,T,CC,CT,TT
rs4821519,G,C,GG,CG,CC
rs1003500,C,T,CC,CT,TT
rs5756405,A,G,AA,AG,GG


In [ ]:
test_genfile_format.get_genotypes_df(rsid_list = ["rs77948203", "rs9610458"])

/Users/ahoang/Documents/Learning/nbdev_tutorial_2nd_try/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,ref,alt,homo_ref,het,homo_alt
rsid,,,,,
rs77948203,G,A,GG,AG,AA
rs9610458,C,T,CC,CT,TT


In [ ]:
assert test_genfile_format.num_samples == 9772

/Users/ahoang/Documents/Learning/nbdev_tutorial_2nd_try/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
#export

def row_vectorize(f):
    @wraps(f)
    def wrapped_f(X, **kwargs):
        rows = X.reshape(-1, X.shape[-1])
        return np.reshape([f(row, **kwargs) for row in rows],
                          X.shape[:-1] + (-1,))
    return wrapped_f


def triplicate_converter(sample_df, *, genotype_df, sample_id, high_lim=.9, low_lim=0.3, NA_val = "NA"):
        """
        takes in allele_list in gen triplicate format and
        return new_list in letter format
        """
        geno_1, geno_2, geno_3 = [sample_df.astype(float)[col] for col in sample_df]
        
        rsid_genotype_df = genotype_df.loc[sample_df.index]
        #all_geno = SimpleNamespace(**test_genfile_format.get_genotypes_df().loc[rsid])
        homo_ref_cond = (geno_1 >=high_lim) & (geno_2 < low_lim) & (geno_3 < low_lim)
        het_cond = (geno_2 >= high_lim) & (geno_1 < low_lim) & (geno_3 < low_lim)
        homo_alt_cond = (geno_3 >= high_lim) & (geno_1 < low_lim) & (geno_2 < low_lim)
        geno_df =  np.select([homo_ref_cond, het_cond, homo_alt_cond],
                 [rsid_genotype_df["homo_ref"], rsid_genotype_df["het"], rsid_genotype_df["homo_alt"]],
                 default = NA_val)
        geno_df = pd.DataFrame(geno_df, index = sample_df.index, columns = [sample_id])
        geno_df.columns.name= "sample_id"
        return geno_df


In [ ]:
first_sample = next(test_genfile_format.sample_columns_iter(rsid_list = ["rs9610458", "rs4821519"]))
first_sample.sample_df

,sample1_1,sample1_2,sample1_3
rsid,,,
rs9610458,0,0.0,1.0
rs4821519,1,0.0,0.0


In [ ]:
first_sample.sample_id

/Users/ahoang/Documents/Learning/nbdev_tutorial_2nd_try/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


'sample1'

In [ ]:
triplicate_converter(first_sample.sample_df,
                     sample_id = first_sample.sample_id,
                    genotype_df = test_genfile_format.get_genotypes_df())

sample_id,sample1
rsid,
rs9610458,TT
rs4821519,GG


In [ ]:
test_genfile_format.apply_func_to_all_samples(triplicate_converter, 
                                              rsid_list = ["rs9610458", "rs4821519"],
                                              genotype_df = test_genfile_format.get_genotypes_df())

/Users/ahoang/Documents/Learning/nbdev_tutorial_2nd_try/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0it [00:00, ?it/s]

sample_id,sample1,sample10,sample100,sample1000,sample1001,sample1002,sample1003,sample1004,sample1005,sample1006,...,sample990,sample991,sample992,sample993,sample994,sample995,sample996,sample997,sample998,sample999
rsid,,,,,,,,,,,,,,,,,,,,,
rs9610458,TT,TT,TT,CT,TT,CC,CT,CT,CT,CC,...,TT,CT,CT,CT,TT,CT,CT,CT,CT,CT
rs4821519,GG,GG,GG,GG,CG,GG,GG,GG,GG,GG,...,GG,GG,GG,GG,GG,CC,GG,GG,GG,GG


If you don't input a list of rsids, it will use all the rsids in the dataset, which could take a **LONG TIME**

In [ ]:
test_geno_each_sample = test_genfile_format.apply_func_to_all_samples(triplicate_converter, 
                                              genotype_df = test_genfile_format.get_genotypes_df()).T # transposed

test_geno_each_sample

/Users/ahoang/Documents/Learning/nbdev_tutorial_2nd_try/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0it [00:00, ?it/s]

rsid,rs77948203,rs1014626,rs9610458,rs5762201,rs1004237,rs134490,rs4821519,rs1003500,rs5756405
sample_id,,,,,,,,,
sample1,GG,TT,TT,GG,CC,NA,GG,CC,AG
sample10,GG,TT,TT,GG,CC,TT,GG,CC,AG
sample100,GG,TT,TT,GG,CC,TT,GG,CC,AG
sample1000,AG,TT,CT,GG,CC,CT,GG,CC,AA
sample1001,GG,TT,TT,GG,CC,CT,CG,CC,AG
...,...,...,...,...,...,...,...,...,...
sample995,GG,TT,CT,GG,CC,CT,CC,CC,AG
sample996,GG,TT,CT,GG,CC,TT,GG,CC,AG
sample997,GG,TT,CT,AG,CC,TT,GG,CC,AG


In [ ]:
catalog.save("geno_each_sample", test_geno_each_sample)

/Users/ahoang/Documents/Learning/nbdev_tutorial_2nd_try/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
test_geno_each_sample[["rs77948203", "rs1014626", "rs1004237"]].reset_index().groupby(["rs77948203", "rs1014626", "rs1004237"])["sample_id"].unique().reset_index() #.melt(value_vars = ["rs77948203", "rs1014626"], ignore_index=False).reset_index().groupby(["rsid", "value"])["index"].unique()

,rs77948203,rs1014626,rs1004237,sample_id
0,AA,TT,CC,"[sample1445, sample1545, sample2271, sample237..."
1,AA,TT,NA,"[sample465, sample6764]"
2,AG,CT,CC,"[sample5663, sample5840]"
3,AG,NA,CC,[sample297]
4,AG,TT,CC,"[sample1000, sample1002, sample1025, sample102..."
5,AG,TT,NA,"[sample6578, sample678]"
6,GG,CT,CC,"[sample1264, sample1359, sample1960, sample220..."
7,GG,NA,CC,"[sample1378, sample1496, sample2247, sample391..."
8,GG,TT,CC,"[sample1, sample10, sample100, sample1001, sam..."
9,GG,TT,NA,"[sample1601, sample2147, sample4016, sample421..."


In [ ]:
test_geno_each_sample

/Users/ahoang/Documents/Learning/nbdev_tutorial_2nd_try/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


rsid,rs77948203,rs1014626,rs9610458,rs5762201,rs1004237,rs134490,rs4821519,rs1003500,rs5756405
sample_id,,,,,,,,,
sample1,GG,TT,TT,GG,CC,NA,GG,CC,AG
sample10,GG,TT,TT,GG,CC,TT,GG,CC,AG
sample100,GG,TT,TT,GG,CC,TT,GG,CC,AG
sample1000,AG,TT,CT,GG,CC,CT,GG,CC,AA
sample1001,GG,TT,TT,GG,CC,CT,CG,CC,AG
...,...,...,...,...,...,...,...,...,...
sample995,GG,TT,CT,GG,CC,CT,CC,CC,AG
sample996,GG,TT,CT,GG,CC,TT,GG,CC,AG
sample997,GG,TT,CT,AG,CC,TT,GG,CC,AG


In [ ]:
test_genfile = test_genfile_format.load()
test_genfile

/Users/ahoang/Documents/Learning/nbdev_tutorial_2nd_try/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,dashes,rsid,position,ref,alt,sample1_1,sample1_2,sample1_3,sample2_1,sample2_2,...,sample9769_3,sample9770_1,sample9770_2,sample9770_3,sample9771_1,sample9771_2,sample9771_3,sample9772_1,sample9772_2,sample9772_3
0,---,rs77948203,21249165,G,A,1,0.000,0.000,1,0.000,...,0,1,0,0,1,0.000,0.000,1,0,0
1,---,rs1014626,21461017,C,T,0,0.000,1.000,0,0.000,...,1,0,0,1,0,0.000,1.000,0,0,1
2,---,rs9610458,22205353,C,T,0,0.000,1.000,0,0.000,...,0,0,1,0,0,1.000,0.000,0,0,1
3,---,rs5762201,27888455,A,G,0,0.000,1.000,0,0.012,...,1,0,0,1,0,0.000,1.000,0,0,1
4,---,rs1004237,28068501,C,T,1,0.000,0.000,1,0.000,...,0,1,0,0,1,0.000,0.000,1,0,0
5,---,rs134490,28730175,C,T,0,0.232,0.768,0,0.014,...,0,0,1,0,0,0.356,0.644,0,0,1
6,---,rs4821519,37102100,G,C,1,0.000,0.000,0,1.000,...,0,1,0,0,1,0.000,0.000,1,0,0
7,---,rs1003500,37262769,C,T,1,0.000,0.000,1,0.000,...,0,1,0,0,1,0.000,0.000,1,0,0
8,---,rs5756405,37310954,A,G,0,1.000,0.000,1,0.000,...,1,0,1,0,1,0.000,0.000,0,1,0


In [ ]:
#export    
class FILE_FORMAT_ENUM(Enum):
    GenFile = GenFileFormat
    
class FILE_TYPE_ENUM(Enum):
    CC = CaseControlFilePathSchema
    S = SingleFilePathSchema
    

/Users/ahoang/Documents/Learning/nbdev_tutorial_2nd_try/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
#export

class OVPDataset(AbstractVersionedDataSet):
    def __init__(self,
                 file_type,
                 file_format,
                 file_path,
                 common_folder=None,
                load_args: Dict[str, Any] = None,
                version: Version = None,
                credentials: Dict[str, Any] = None,
                fs_args: Dict[str, Any] = None,
                ):
        
        self._file_path_class = FILE_TYPE_ENUM[file_type].value
        self._file_path = self._file_path_class(**file_path) if common_folder is None else self._file_path_class(**file_path, common_folder = common_folder) #custom file path
        
        
        self._version = version
        
        self._file_format_class = FILE_FORMAT_ENUM[file_format].value
        #self._file_format = self._file_format_class(**load_args)
        
        self.files = SimpleNamespace(**{single_file : self._file_format_class(filepath = single_file_path, **load_args)\
                                        for single_file, single_file_path in self._file_path.full_file_path.items()})#self._file_path.apply_func(self._file_format_class, **load_args)
        
        _fs_args = deepcopy(fs_args) or {}
        _fs_open_args_load = _fs_args.pop("open_args_load", {})
        _fs_open_args_save = _fs_args.pop("open_args_save", {})
        _credentials = deepcopy(credentials) or {}
        
        #protocol, path = get_protocol_and_path(filepath, version)
        if self._file_path.protocol == "file":
            _fs_args.setdefault("auto_mkdir", True)
        
        _fs_open_args_save.setdefault("mode", "w")
        self._fs_open_args_load = _fs_open_args_load
        self._fs_open_args_save = _fs_open_args_save
        
        self._protocol = self._file_path.protocol
        self._fs = fsspec.filesystem(self._protocol, **_credentials, **_fs_args)

#         self._protocol = protocol
#         self._fs = fsspec.filesystem(self._protocol, **_credentials, **_fs_args)
    
    
    @property
    def full_file_path(self):
        return self._file_path.full_file_path
    
    def _load(self):
        return self
    
    def _save(self):
        pass
    
    def _describe(self):
        pass

In [ ]:
test_conf_catalog = conf_catalog["genetic_file"].copy()
test_conf_catalog.pop("type")

'nbdev_tutorial.datasets.genetic_datasets.GenFileDataset'

In [ ]:
test_genetic_dataset = OVPDataset(**test_conf_catalog)
test_genetic_dataset

In [ ]:
test_case_file = test_genetic_dataset.files.case
test_control_file = test_genetic_dataset.files.control

In [ ]:
test_case_file.get_genotypes_df()

,ref,alt,homo_ref,het,homo_alt
rsid,,,,,
rs77948203,G,A,GG,AG,AA
rs1014626,C,T,CC,CT,TT
rs9610458,C,T,CC,CT,TT
rs5762201,A,G,AA,AG,GG
rs1004237,C,T,CC,CT,TT
rs134490,C,T,CC,CT,TT
rs4821519,G,C,GG,CG,CC
rs1003500,C,T,CC,CT,TT
rs5756405,A,G,AA,AG,GG


In [ ]:
test_control_file.get_genotypes_df()

/Users/ahoang/Documents/Learning/nbdev_tutorial_2nd_try/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,ref,alt,homo_ref,het,homo_alt
rsid,,,,,
rs77948203,G,A,GG,AG,AA
rs1014626,C,T,CC,CT,TT
rs9610458,C,T,CC,CT,TT
rs5762201,A,G,AA,AG,GG
rs1004237,C,T,CC,CT,TT
rs134490,C,T,CC,CT,TT
rs4821519,G,C,GG,CG,CC
rs1003500,C,T,CC,CT,TT
rs5756405,A,G,AA,AG,GG


In [ ]:
case_all_geno_file = test_case_file.apply_func_to_all_samples(triplicate_converter, 
                                              genotype_df = test_case_file.get_genotypes_df()).T
case_all_geno_file

/Users/ahoang/Documents/Learning/nbdev_tutorial_2nd_try/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0it [00:00, ?it/s]

rsid,rs77948203,rs1014626,rs9610458,rs5762201,rs1004237,rs134490,rs4821519,rs1003500,rs5756405
sample_id,,,,,,,,,
sample1,GG,TT,TT,GG,CC,NA,GG,CC,AG
sample10,GG,TT,TT,GG,CC,TT,GG,CC,AG
sample100,GG,TT,TT,GG,CC,TT,GG,CC,AG
sample1000,AG,TT,CT,GG,CC,CT,GG,CC,AA
sample1001,GG,TT,TT,GG,CC,CT,CG,CC,AG
...,...,...,...,...,...,...,...,...,...
sample995,GG,TT,CT,GG,CC,CT,CC,CC,AG
sample996,GG,TT,CT,GG,CC,TT,GG,CC,AG
sample997,GG,TT,CT,AG,CC,TT,GG,CC,AG


In [ ]:
control_all_geno_file = test_control_file.apply_func_to_all_samples(triplicate_converter, 
                                              genotype_df = test_control_file.get_genotypes_df()).T
control_all_geno_file

/Users/ahoang/Documents/Learning/nbdev_tutorial_2nd_try/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0it [00:00, ?it/s]

rsid,rs77948203,rs1014626,rs9610458,rs5762201,rs1004237,rs134490,rs4821519,rs1003500,rs5756405
sample_id,,,,,,,,,
sample1,GG,TT,TT,AG,CC,CT,GG,CC,AG
sample10,GG,TT,CT,GG,CC,TT,GG,CC,AA
sample100,GG,TT,CT,GG,CC,NA,GG,CC,AG
sample1000,AG,TT,TT,GG,CC,CT,GG,CC,AA
sample1001,GG,TT,CC,GG,CC,CT,GG,CC,AG
...,...,...,...,...,...,...,...,...,...
sample995,GG,TT,TT,GG,CC,TT,GG,CC,AG
sample996,GG,TT,CT,GG,CC,TT,GG,CC,AA
sample997,GG,TT,CC,GG,CC,NA,GG,CC,AG


In [ ]:
assert not case_all_geno_file.equals(control_all_geno_file)

In [ ]:
catalog.save("case_geno_each_sample", case_all_geno_file)
catalog.load("case_geno_each_sample")

,rs77948203,rs1014626,rs9610458,rs5762201,rs1004237,rs134490,rs4821519,rs1003500,rs5756405
sample_id,,,,,,,,,
sample1,GG,TT,TT,GG,CC,NaN,GG,CC,AG
sample10,GG,TT,TT,GG,CC,TT,GG,CC,AG
sample100,GG,TT,TT,GG,CC,TT,GG,CC,AG
sample1000,AG,TT,CT,GG,CC,CT,GG,CC,AA
sample1001,GG,TT,TT,GG,CC,CT,CG,CC,AG
...,...,...,...,...,...,...,...,...,...
sample995,GG,TT,CT,GG,CC,CT,CC,CC,AG
sample996,GG,TT,CT,GG,CC,TT,GG,CC,AG
sample997,GG,TT,CT,AG,CC,TT,GG,CC,AG


In [ ]:
catalog.save("control_geno_each_sample", control_all_geno_file)
catalog.load("control_geno_each_sample")

/Users/ahoang/Documents/Learning/nbdev_tutorial_2nd_try/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,rs77948203,rs1014626,rs9610458,rs5762201,rs1004237,rs134490,rs4821519,rs1003500,rs5756405
sample_id,,,,,,,,,
sample1,GG,TT,TT,AG,CC,CT,GG,CC,AG
sample10,GG,TT,CT,GG,CC,TT,GG,CC,AA
sample100,GG,TT,CT,GG,CC,NaN,GG,CC,AG
sample1000,AG,TT,TT,GG,CC,CT,GG,CC,AA
sample1001,GG,TT,CC,GG,CC,CT,GG,CC,AG
...,...,...,...,...,...,...,...,...,...
sample995,GG,TT,TT,GG,CC,TT,GG,CC,AG
sample996,GG,TT,CT,GG,CC,TT,GG,CC,AA
sample997,GG,TT,CC,GG,CC,NaN,GG,CC,AG


In [ ]:
#export

# def G_O_dict_maker_single(self, filename):
#         """
#         creates GWAS and outside genetic info data structures for a single file
#         """
#         file = open(filename, "r")

#         found_GWAS = set()
#         found_outside = set()
#         GWAS_map = {}
#         outside_map = {}

#         line_num = 1

#         for i,line in tqdm(enumerate(file)):
#             rsid = self.column_getter(line, self.rsid_col)
#             if rsid in self.GWAS_set or rsid in self.outside_set:
#                 pos_1 = self.column_getter(line, self.snp_col)
#                 if "/" in pos_1:
#                     pos_2 = pos_1[2]
#                     pos_1 = pos_1[0]
#                 else:
#                     pos_2 = self.column_getter(line, self.snp_col + 1)
#                 if len(pos_1) > 1 or len(pos_2) > 1:
#                     continue

#                 poss_geno = possible_genotypes(pos_1, pos_2)
#                 head, alleles = self.column_cutter(line, self.col_cut)
#                 allele_list = alleles.split(self.delim)
#                 if self.is_triples:
#                     if len(allele_list) % 3 != 0:
#                         raise ValueError(
#                             "Missing data. Incomplete genotype triple found in line " + str(line_num) + ".")
#                     else:
#                         allele_list = self.triplicate_converter(
#                             allele_list, pos_1, pos_2)

#                 allele_list = list(map(self.clean_item_cached, allele_list))
#                 # remove list here
#                 total = len(list(allele_list))

#                 if rsid in self.GWAS_set:
#                     found_GWAS.add(rsid)
#                     new_map = defaultdict(list)
#                     new_map["alleles"] = poss_geno
#                     new_map["total"] = total
#                     for i in range(len(list(allele_list))):
#                         curr_allele = allele_list[i]
#                         if curr_allele != self.NA:
#                             new_map[curr_allele].append(i)
#                         else:
#                             new_map["NA_indices"].append(i)
#                     GWAS_map[rsid] = new_map
#                 else:
#                     found_outside.add(rsid)
#                     outside_map[rsid] = allele_list, poss_geno
#             line_num += 1
#         file.close()

#         logging.info(str(len(found_GWAS)) + " out of " + str(len(self.GWAS_set)
#                                                              ) + " GWAS rsIDs found in " + filename + ".")
#         not_found_GWAS = self.GWAS_set.difference(found_GWAS)
#         if len(not_found_GWAS) > 0:
#             logging.info("WARNING: The following GWAS rsIDs are found in the pairing file, but not " +
#                          filename + ".\n" + str(not_found_GWAS))
#             if not_found_GWAS == self.GWAS_set:
#                 return None, None

#         logging.info(str(len(found_outside)) + " out of " + str(len(self.outside_set)
#                                                                 ) + " outside rsIDs found in " + filename + ".")
#         not_found_outside = self.outside_set.difference(found_outside)
#         if len(not_found_outside):
#             logging.info("WARNING: The following outside rsIDs are found in the pairing file, but not " +
#                          filename + ".\n" + str(not_found_outside))
#             if not_found_outside == self.outside_set:
#                 return None, None

#         df_data = [*zip(not_found_GWAS, ["not_found"] * len(not_found_GWAS), ["GWAS"]* len(not_found_GWAS)),
#             *zip(not_found_outside, ["not_found"] * len(not_found_outside), ["outside"]* len(not_found_outside)),]

#         self.filter_df = pd.DataFrame(df_data, columns=["rsID", "reason_for_filtering", "SNP_type"])

#         return GWAS_map, outside_map

In [ ]:
#export
def say_hello(to):
    "Say hello to somebody"
    return f'Hello {to}!'

In [ ]:
say_hello("Sylvain")

'Hello Sylvain!'

In [ ]:
from IPython.display import display,SVG
display(SVG('<svg height="100"><circle cx="50" cy="50" r="40"/></svg>'))

In [ ]:
assert say_hello("Jeremy")=="Hello Jeremy!"